In [45]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from math import factorial
import numpy as np
from scipy import stats as st

In [46]:
visitas = pd.read_csv('datos/visits_log_us.csv')

In [47]:
pedidos = pd.read_csv('datos/orders_log_us.csv')

In [48]:
gastos = pd.read_csv('datos/costs_us.csv')

VISITAS 

In [49]:
visitas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359400 entries, 0 to 359399
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Device     359400 non-null  object
 1   End Ts     359400 non-null  object
 2   Source Id  359400 non-null  int64 
 3   Start Ts   359400 non-null  object
 4   Uid        359400 non-null  uint64
dtypes: int64(1), object(3), uint64(1)
memory usage: 13.7+ MB


In [50]:
visitas.head()

,Device,End Ts,Source Id,Start Ts,Uid
0,touch,2017-12-20 17:38:00,4,2017-12-20 17:20:00,16879256277535980062
1,desktop,2018-02-19 17:21:00,2,2018-02-19 16:53:00,104060357244891740
2,touch,2017-07-01 01:54:00,5,2017-07-01 01:54:00,7459035603376831527
3,desktop,2018-05-20 11:23:00,9,2018-05-20 10:59:00,16174680259334210214
4,desktop,2017-12-27 14:06:00,3,2017-12-27 14:06:00,9969694820036681168


In [63]:
porcentaje_nulos = visitas.isnull().mean() * 100
print(porcentaje_nulos)

Device       0.0
End Ts       0.0
Source Id    0.0
Start Ts     0.0
Uid          0.0
dtype: float64


In [64]:
visitas = visitas.drop_duplicates()
visitas = visitas.reset_index(drop=True)


In [65]:
visitas.head()

,Device,End Ts,Source Id,Start Ts,Uid
0,touch,2017-12-20 17:38:00,4,2017-12-20 17:20:00,16879256277535980062
1,desktop,2018-02-19 17:21:00,2,2018-02-19 16:53:00,104060357244891740
2,touch,2017-07-01 01:54:00,5,2017-07-01 01:54:00,7459035603376831527
3,desktop,2018-05-20 11:23:00,9,2018-05-20 10:59:00,16174680259334210214
4,desktop,2017-12-27 14:06:00,3,2017-12-27 14:06:00,9969694820036681168


Convertiremos las columnas 'Start Ts' y 'End Ts' a formato datetime para poder trabajar con ellas

In [67]:
visitas['Start Ts'] = pd.to_datetime(visitas['Start Ts'])
visitas['End Ts'] = pd.to_datetime(visitas['End Ts'])
print(visitas[['Start Ts', 'End Ts']].dtypes)


Start Ts    datetime64[ns]
End Ts      datetime64[ns]
dtype: object


Ahora Trabajaremos para encontrar la cantidad de usuarios por día, semana, mes

Revisaremos cuantas sesiones se realizan por día.

Revisaremos si un usuario ingresa más de una vez

Vamos a revisar cuanto dura cada sesión del visitante

Revisaremos con que frecuencia regresa al sitio cada usuario

Crearemos un gráfico

PEDIDOS

In [55]:
pedidos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50415 entries, 0 to 50414
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Buy Ts   50415 non-null  object 
 1   Revenue  50415 non-null  float64
 2   Uid      50415 non-null  uint64 
dtypes: float64(1), object(1), uint64(1)
memory usage: 1.2+ MB


In [56]:
pedidos.head(5)

,Buy Ts,Revenue,Uid
0,2017-06-01 00:10:00,17.00,10329302124590727494
1,2017-06-01 00:25:00,0.55,11627257723692907447
2,2017-06-01 00:27:00,0.37,17903680561304213844
3,2017-06-01 00:29:00,0.55,16109239769442553005
4,2017-06-01 07:58:00,0.37,14200605875248379450


In [57]:
print(pedidos.isnull().sum())

Buy Ts     0
Revenue    0
Uid        0
dtype: int64


In [58]:
pedidos = pedidos.drop_duplicates()
print(pedidos.duplicated().sum())
pedidos.reset_index(drop=True)

0


,Buy Ts,Revenue,Uid
0,2017-06-01 00:10:00,17.00,10329302124590727494
1,2017-06-01 00:25:00,0.55,11627257723692907447
2,2017-06-01 00:27:00,0.37,17903680561304213844
3,2017-06-01 00:29:00,0.55,16109239769442553005
4,2017-06-01 07:58:00,0.37,14200605875248379450
...,...,...,...
50410,2018-05-31 23:50:00,4.64,12296626599487328624
50411,2018-05-31 23:50:00,5.80,11369640365507475976
50412,2018-05-31 23:54:00,0.30,1786462140797698849
50413,2018-05-31 23:56:00,3.67,3993697860786194247


Haremos un Histograma por mes

Convertiremos la columna 'Buy Ts' a formato datetime para poder trabajar con ella

In [59]:
pedidos['Buy Ts'] = pd.to_datetime(pedidos['Buy Ts'])

Vamos a obtener la primera Orden

In [60]:
primera_orden = pedidos.groupby('Uid')['Buy Ts'].min()
print(primera_orden.head())

Uid
313578113262317    2018-01-03 21:51:00
1575281904278712   2017-06-03 10:13:00
2429014661409475   2017-10-11 18:33:00
2464366381792757   2018-01-28 15:54:00
2551852515556206   2017-11-24 10:14:00
Name: Buy Ts, dtype: datetime64[ns]


Vamos a obtener

In [61]:
primera_orden = primera_orden.to_frame(name='primera_compra')
pedidos = pedidos.merge(primera_orden, left_on='Uid', right_index=True, how='left')
print(pedidos.head())

               Buy Ts  Revenue                   Uid      primera_compra
0 2017-06-01 00:10:00    17.00  10329302124590727494 2017-06-01 00:10:00
1 2017-06-01 00:25:00     0.55  11627257723692907447 2017-06-01 00:25:00
2 2017-06-01 00:27:00     0.37  17903680561304213844 2017-06-01 00:27:00
3 2017-06-01 00:29:00     0.55  16109239769442553005 2017-06-01 00:29:00
4 2017-06-01 07:58:00     0.37  14200605875248379450 2017-06-01 07:58:00


Vamos a trabajar para obtener los cohortes Mensual

In [62]:
pedidos['primera_orden_mes'] = pedidos['primera_compra'].astype('datetime64[M]')
pedidos['orden_mes'] = pedidos['Buy Ts'].astype('datetime64[M]')

print(pedidos.head())

TypeError: Cannot cast DatetimeArray to dtype datetime64[M]

In [ ]:
grupo_cohort = pedidos.groupby('primera_orden_mes').agg({'Uid': 'nunique', 'Revenue': 'sum'})
print(grupo_cohort.head())

KeyError: 'primera_orden_mes'

Podemos observar que los ingresos por cohortes bajaron, se recuperaron pero nuevamente bajaron y otra vez hay un repunte

Realizaremos una tabla Dinámica para conocer que clientes siguen realizando compras despues de que realizaron la primera

In [ ]:
pedidos.pivot_table(
    index='primera_orden_mes',
    columns='orden_mes',
    values='Uid',
    aggfunc='nunique',)

KeyError: 'primera_orden_mes'

Existe una disminución de clientes despues del primer mes, luego se recupera y nuevamente hay una caida y otra vez se recupera.

Haremos un gráfico de lineas

Vamos a calcular el promedio de compra(ingresos)

In [ ]:
pedidos_agrupados_cohorts = pedidos.groupby( ['primera_orden_mes', 'orden_mes']).agg({'Revenue': 'sum', 'Uid': 'nunique'})
print(pedidos_agrupados_cohorts.head())

KeyError: 'primera_orden_mes'

In [ ]:
pedidos_agrupados_cohorts.columns

Buscaremos la compra por usuario

In [ ]:
pedidos_agrupados_cohorts['Ingreso_por_uid'] = (pedidos_agrupados_cohorts['Revenue']
    / pedidos_agrupados_cohorts['Uid'])

Realizaremos una Tabla Dinámica para observar la compra mensual por usuario.

In [ ]:
pedidos_agrupados_cohorts.pivot_table(
    index='primera_orden_mes',
    columns='orden_mes',
    values='Ingreso_por_uid',
    aggfunc='mean',)

NameError: name 'pedidos_agrupados_cohorts' is not defined